# 🏍️ LossZero: Motorcycle Night Ride SegFormer-B2 Optimized

이 노트북은 **SegFormer-B2** 모델을 사용하여 야간 오토바이 주행 이미지의 시멘틱 세그멘테이션을 수행합니다.

### 🛠️ 주요 시나리오
- **모델**: SegFormer-B2 (Transformer 기반)
- **백본**: MiT-B2
- **사전 학습**: Cityscapes (도로 환경 특화)
- **최적화**: AdamW + FP16 Mixed Precision
- **손실 함수**: Weighted CrossEntropy (중요 객체 가중치 부여)

In [201]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb
wandb.login()

print(f"PyTorch version: {torch.__version__}")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


PyTorch version: 2.6.0


## Colab 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [216]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"

    return "cpu"

def num_worker():
    if torch.cuda.is_available():
        return os.cpu_count()

    return 0

# ⚙️ 설정 (Configuration)
# DATA_DIR = "/content/drive/MyDrive/motor_model"
DATA_DIR = "/home/jovyan/work/motorcycle/Dataset"
# DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset")
print("Detected Local Environment")
CFG = {
    "project": "LossZero",
    "model_name": "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",
    "img_size": (384, 672),
    "batch_size": 8,
    "lr": 1e-4,
    "epochs": 20,
    "device": get_device(),
    "num_worker": num_worker()
}

print(f"Using device: {CFG['device']}")
print(f"Data directory: {DATA_DIR}")

Detected Local Environment
Using device: cpu
Data directory: /Users/jamesyang/Projects/LossZero/data/Motorcycle Night Ride Dataset


In [217]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        cat_id = ann['category_id']
        if cat_id in id_to_idx:
            cls_idx = id_to_idx[cat_id]
            pixel_mask = coco.annToMask(ann)
            mask[pixel_mask == 1] = cls_idx

    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(img_dir, img_info['file_name'])

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)

    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']

    return image, torch.as_tensor(mask).long()

train_transform = A.Compose([
    #  원본 해상도에서 480x480 크기로 무작위 추출 (화질 저하 없음)
    A.RandomCrop(height=CFG['img_size'][0], width=CFG['img_size'][1], p=1.0),
    A.PadIfNeeded(min_height=CFG['img_size'][0], min_width=CFG['img_size'][1], p=1.0),

    # --- 야간 전용 Augmentation 추가 ---
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5), # 어두운 저조도 개선
    A.GaussNoise(std_range=(0.02, 0.05), p=0.3), # 야간 노이즈 대응

    # --- 기하학적 변형 (데이터 수 보충용) ---
    A.HorizontalFlip(p=0.5), # 좌우 반전
    # 0.0625는 머신러닝/딥러닝 커뮤니티에서 오랫동안 검증된 '사실상 표준(De Facto Standard)
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=25, p=0.5), # 이동/크기/회전

    # ImageNet 데이타셋의 평균값 나쁘지 않음. SegFormer가 ImageNet/Cityscapes로 배웠으니까
    # 모델이 새로운 사진을 받을 때: 입력_이미지 = (원본_이미지 - 평균) / 표준편차
    # 이렇게 계산해주면, 어떤 사진이 들어와도 "평균이 0이고 표준편차가 1인(Standard Normal Distribution)" 아주 예쁜 데이터로 변신
    # 전체 야간 데이터셋의 Mean/Std를 직접 계산한 값
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])

coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys())
cat_ids = coco.getCatIds()
id_to_idx = {cat_id: i for i, cat_id in enumerate(cat_ids)}
print(f"Category Mapping: {id_to_idx}")

loading annotations into memory...
Done (t=1.08s)
creating index...
index created!
Category Mapping: {1329681: 0, 1323885: 1, 1323884: 2, 1323882: 3, 1323881: 4, 1323880: 5}


## Traing / Val 분리

In [218]:
import random
from sklearn.model_selection import train_test_split
class MotorcycleNightRideDataset(Dataset):
    def __init__(self, coco, img_ids, img_dir, id_to_idx, transform=None, use_copy_paste=False):
        self.coco = coco
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.id_to_idx = id_to_idx
        self.transform = transform
        self.use_copy_paste = use_copy_paste  # Copy-Paste 활성화 여부 (Train만 True)

    def __len__(self):
        return len(self.img_ids)

    def load_image_mask(self, idx):
        """인덱스에 해당하는 이미지와 마스크를 로드하고 BGR->RGB 변환"""
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_dir, img_info['file_name'])

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 마스크 생성 (기존 함수 활용)
        mask = create_mask_from_json(self.coco, img_id, img_info, self.id_to_idx)
        return image, mask

    def apply_copy_paste(self, image, mask):
        """
        Copy-Paste Augmentation:
        다른 이미지(Donor)에서 'Lane Mark(3)'나 'Moveable(2)' 같은 소수 클래스를 오려내어
        현재 이미지(Target)에 붙여넣습니다.
        """
        # 1. 기증자(Donor) 무작위 선택
        donor_idx = random.randint(0, len(self.img_ids) - 1)
        donor_img, donor_mask = self.load_image_mask(donor_idx)

        # 2. 오려낼 타겟 클래스 정의 (차선과 움직이는 물체 집중 공략)
        # Lane Mark: 3, Moveable: 2
        target_indices = [2, 3]

        # donor_mask에서 해당 클래스인 픽셀만 True (나머지 False)
        # np.isin을 사용하여 여러 클래스를 한 번에 처리
        paste_mask = np.isin(donor_mask, target_indices)

        # 3. 붙여넣기 (Paste)
        # 해당 영역에 내용물이 있을 경우에만 실행
        if np.any(paste_mask):
            # 이미지 덮어쓰기
            image[paste_mask] = donor_img[paste_mask]
            # 마스크 덮어쓰기 (정답지 수정)
            mask[paste_mask] = donor_mask[paste_mask]

        return image, mask

    def __getitem__(self, idx):
        # 1. 원본 데이터 로드
        image, mask = self.load_image_mask(idx)

        # 2. Copy-Paste 적용 (훈련 데이터셋이고, 50% 확률 당첨 시)
        if self.use_copy_paste and random.random() < 0.5:
            image, mask = self.apply_copy_paste(image, mask)

        # 3. Albumentations 변환 (Resize, ColorJitter, Normalization 등)
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, torch.as_tensor(mask).long()

# --------------------------------------------------------------------------
# 1. 데이터 로드 및 ID 분할 (7:2:1)
coco = COCO(JSON_PATH)
all_ids = list(coco.imgs.keys())

# First split: 70% train, 30% temp
train_ids, temp_ids = train_test_split(all_ids, test_size=0.3, random_state=42)
# Second split: temp_ids into 2/3 for val (0.2 of total), 1/3 for test (0.1 of total)
val_ids, test_ids = train_test_split(temp_ids, test_size=1/3, random_state=42)

# 2. Transform 정의 (기존 정의 활용)
val_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])
# Test transform is the same as validation transform
test_transform = val_transform

# 3. 데이터셋 인스턴스 생성 (★Train에만 Copy-Paste 활성화★)
train_dataset = MotorcycleNightRideDataset(
    coco, train_ids, IMG_DIR, id_to_idx,
    transform=train_transform,
    use_copy_paste=True  # Copy-Paste ON!
)
val_dataset = MotorcycleNightRideDataset(
    coco, val_ids, IMG_DIR, id_to_idx,
    transform=val_transform,
    use_copy_paste=False # Val에는 절대 쓰면 안 됨 (순수 평가)
)
test_dataset = MotorcycleNightRideDataset( # New test dataset
    coco, test_ids, IMG_DIR, id_to_idx,
    transform=test_transform,
    use_copy_paste=False # Test에는 절대 쓰면 안 됨 (순수 평가)
)

# 4. 데이터 로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['batch_size'],
    shuffle=True,
    num_workers=CFG['num_worker'],
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['batch_size'],
    shuffle=False,
    num_workers=CFG['num_worker'],
    pin_memory=True
)
test_loader = DataLoader( # New test loader
    test_dataset,
    batch_size=CFG['batch_size'],
    shuffle=False,
    num_workers=CFG['num_worker'],
    pin_memory=True
)

# Define CFG_EVAL for later use
CFG_EVAL = {
    "num_classes": len(id_to_idx)
}

print(f"✅ Data Ready with Copy-Paste Augmentation!")
print(f"   Train={len(train_ids)} (CP Active), Val={len(val_ids)}, Test={len(test_ids)}")

loading annotations into memory...
Done (t=0.95s)
creating index...
index created!
✅ Data Ready with Copy-Paste Augmentation!
   Train=140 (CP Active), Val=40, Test=20


### 📉 클래스별 분포 요약 (내림차순)

1. **Undrivable (주행 불가 영역)**: **42.9%** (압도적 1위)
   - 배경(하늘, 건물, 풀숲 등)이 이미지의 절반 가까이 차지합니다.
2. **Road (주행 가능 도로)**: **27.1%**
   - 도로 자체도 꽤 많은 영역을 차지합니다.
3. **My bike (내 오토바이)**: **15.8%**
   - 주행자 시점이라 내 오토바이가 항상 보이기 때문에 비율이 높습니다.
4. **Rider (탑승자)**: **8.1%**
   - 다른 오토바이 운전자나 내 신체가 포함된 것으로 보입니다.
5. **Moveable (이동 물체)**: **4.7%**
   - 다른 차량, 보행자 등 안전에 가장 중요한 장애물인데 비율이 매우 낮습니다.
6. **Lane Mark (차선)**: **1.4%**
   - 가장 심각한 불균형입니다. 도로 주행의 핵심인 차선이 고작 1% 남짓입니다.

In [219]:
id2label = {i: coco.loadCats(cat_id)[0]['name'] for cat_id, i in id_to_idx.items()}
label2id = {v: k for k, v in id2label.items()}

model = SegformerForSemanticSegmentation.from_pretrained(
    CFG['model_name'],
    num_labels=len(id_to_idx),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(CFG['device'])

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CFG['lr'], # Learning Rate
    weight_decay=0.05 #가중치의 관성 제어, 옵티마이저에서 0.01이라는 값은 매 학습 단계(Step)마다 현재 가중치 값을 얼마나 깎을지를 결정하는 비율입니다.
)

# ⚖️ 클래스별 가중치 설정 (Class Weights)
weights = torch.tensor([
    3.0,   # Rider: 5.0 → 3.0 (중요하지만 과하지 않게)
    1.5,   # My bike: 2.0 → 1.5 (내 오토바이는 너무 잘 맞히니 조금 더 낮춤)
    6.0,   # Moveable: 10.0 → 6.0 (절반으로 줄여서 부담 완화)
    12.0,  # Lane Mark: 20.0 → 12 (여전히 제일 강력하지만, 20배는 너무 가혹했음)
    1.0,   # Road: 1.0 (기준점 유지)
    0.8    # Undrivable: 0.5 → 0.8 (배경을 너무 무시해서 도로 경계가 무너지는 것 방지)
], dtype=torch.float).to(CFG['device'])

criterion = nn.CrossEntropyLoss(weight=weights)

scaler = GradScaler('cuda') if CFG['device'] == 'cuda' else None

Loading weights:   0%|          | 0/380 [00:00<?, ?it/s]

SegformerForSemanticSegmentation LOAD REPORT from: nvidia/segformer-b2-finetuned-cityscapes-1024-1024
Key                           | Status   |                                                                                                    
------------------------------+----------+----------------------------------------------------------------------------------------------------
decode_head.classifier.bias   | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19]) vs model:torch.Size([6])                      
decode_head.classifier.weight | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19, 768, 1, 1]) vs model:torch.Size([6, 768, 1, 1])

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [221]:
import torch
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix

# [1] 기본 IoU 계산 로직 (Category-specific)
def compute_category_iou(preds, targets, num_classes):
    preds_flat = preds.flatten().cpu().numpy()
    targets_flat = targets.flatten().cpu().numpy()
    valid_mask = (targets_flat >= 0) & (targets_flat < num_classes)
    preds_flat = preds_flat[valid_mask]
    targets_flat = targets_flat[valid_mask]
    cm = confusion_matrix(targets_flat, preds_flat, labels=range(num_classes))
    intersection = np.diag(cm)
    ground_truth_set = cm.sum(axis=1)
    predicted_set = cm.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    iou = intersection / (union + 1e-6)
    return iou

# [2] Boundary IoU 계산 로직
def get_boundary(mask, dilation_pixels=2):
    mask = mask.astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    eroded = cv2.erode(mask, kernel, iterations=dilation_pixels)
    boundary = mask - eroded
    return boundary

def compute_boundary_iou(preds, targets, num_classes, dilation_pixels=2):
    preds_np = preds.cpu().numpy()
    targets_np = targets.cpu().numpy()
    b_ious = []
    for c in range(num_classes):
        class_preds = (preds_np == c)
        class_targets = (targets_np == c)
        ious_per_batch = []
        for i in range(preds_np.shape[0]):
            gt_boundary = get_boundary(class_targets[i], dilation_pixels)
            pred_boundary = get_boundary(class_preds[i], dilation_pixels)
            intersection = ((gt_boundary > 0) & (pred_boundary > 0)).sum()
            union = ((gt_boundary > 0) | (pred_boundary > 0)).sum()
            if union == 0:
                ious_per_batch.append(1.0)
            else:
                ious_per_batch.append(intersection / union)
        if not ious_per_batch:
            b_ious.append(0.0)
        else:
            b_ious.append(np.mean(ious_per_batch))
    return np.array(b_ious)

# [3] 통합 평가 함수 (Metrics Calculation)
def evaluate_metrics(preds, targets, num_classes):
    """예측값과 정답을 받아 모든 지표(Category IoU, Boundary IoU)를 한 번에 계산"""
    cat_iou = compute_category_iou(preds, targets, num_classes)
    bound_iou = compute_boundary_iou(preds, targets, num_classes)
    return cat_iou, bound_iou

# [4] 상세 리포트 출력 함수 (Report Generation)
def print_evaluation_report(avg_cat_iou, avg_bound_iou, id2label=None):
    """계산된 평균 IoU 값들을 받아 보기 좋게 출력"""
    print("\n[Validation Report]")
    print("  Category-specific IoU:")
    for i, iou in enumerate(avg_cat_iou):
        label = id2label[i] if id2label else f"Class {i}"
        print(f"    - {label}: {iou:.4f}")

    print("\n  Boundary IoU (Details):")
    for i, iou in enumerate(avg_bound_iou):
        label = id2label[i] if id2label else f"Class {i}"
        print(f"    - {label}: {iou:.4f}")

    mIoU = np.nanmean(avg_cat_iou)
    mBoU = np.nanmean(avg_bound_iou)
    print(f"\n  --> mIoU: {mIoU:.4f} | mBoU: {mBoU:.4f}\n")
    return mIoU, mBoU


In [222]:
# [공통 로직] 데이터 전송, 추론, 확대, 손실 계산
def forward_step(model, images, masks, criterion, device):
    # 데이터를 GPU(CUDA) 또는 CPU 중 실제 연산이 일어날 장치로 보낸다
    # torch.Tensor 형태이며 contiguous()로 메모리를 정렬한다
    X = images.to(device).contiguous()
    y = masks.to(device).contiguous()

    # Forward Pass
    # logits: 우리가 찾는 클래스별 점수판 (필수!) -10.5나 15.2 같은 자유로운 숫자
    outputs = model(X).logits

    # 확대 (Interpolation)
    # 모델 결과물(outputs)은 연산 효율을 위해 96x96으로 축소되어 있음
    # 이를 정답지 y와 똑같은 크기(384x384)로 부드럽게 확대(Interpolate)
    # y.shape[-2:] -> (384, 384)
    upsampled_logits = nn.functional.interpolate(
        outputs, size=y.shape[-2:], mode="bilinear", align_corners=False
    )

    # 오차(Loss) 계산
    loss = criterion(upsampled_logits, y)

    return loss, upsampled_logits, y

# [학습 단계] 한 에폭 동안의 학습 집행
def train_one_epoch(model, loader, optimizer, criterion, device, scaler=None, epoch=0):
    model.train()
    train_loss_sum = 0
    pbar = tqdm(loader, desc=f"Epoch {epoch+1} [Train]")

    for images, masks in pbar:
        optimizer.zero_grad()

        # Mixed Precision 지원 (CUDA 전용)
        if device == 'cuda' and scaler:
            with torch.amp.autocast('cuda'):
                loss, _, _ = forward_step(model, images, masks, criterion, device)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            # 역전파 및 가중치 업데이트
            loss, _, _ = forward_step(model, images, masks, criterion, device)
            loss.backward()
            optimizer.step()

        # 모델이 예측한 값과 정답 사이의 거리 기록
        train_loss_sum += loss.item()
        pbar.set_postfix(Loss=f"{loss.item():.4f}")

    return train_loss_sum / len(loader)

# [검증 단계] 모듈화된 평가 로직 적용
def validate(model, loader, criterion, device, num_classes=6, id2label=None):
    model.eval()
    val_loss_sum = 0

    # 지표 누적용 배열 (Total Accumulators)
    total_cat_ious = np.zeros(num_classes)
    total_bound_ious = np.zeros(num_classes)

    with torch.no_grad():
        for images, masks in loader:
            # 1. 공통 로직 실행 (Loss 계산)
            loss, logits, y = forward_step(model, images, masks, criterion, device)
            val_loss_sum += loss.item()

            # 2. 예측값 변환
            preds = torch.argmax(logits, dim=1)

            # 3. 통합 평가 모듈 호출
            cat_iou, bound_iou = evaluate_metrics(preds, y, num_classes)
            total_cat_ious += cat_iou
            total_bound_ious += bound_iou

    # 에폭 평균 계산
    avg_loss = val_loss_sum / len(loader)
    avg_cat_iou = total_cat_ious / len(loader)
    avg_bound_iou = total_bound_ious / len(loader)

    # 4. 상세 리포트 출력 모듈 호출
    mIoU, mBoU = print_evaluation_report(avg_cat_iou, avg_bound_iou, id2label)

    return avg_loss, mIoU, mBoU, avg_cat_iou, avg_bound_iou



In [ ]:
# --- 🚀 메인 학습 루프 (Auto-Save 기능 탑재) ---
print("🚀 SegFormer-B2 Training Start with Dual Auto-Save Strategy...")

wandb.init(project=CFG['project'], config=CFG)

best_miou = 0.0
best_mbou = 0.0
best_miou_epoch = 0

save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(CFG['epochs']):
    # 1. 학습 실행
    avg_train_loss = train_one_epoch(model, train_loader, optimizer, criterion, CFG['device'], scaler, epoch)

    # 2. 검증 실행
    avg_val_loss, avg_val_miou, avg_val_mbou, mean_cat_ious, mean_bound_ious = validate(model, val_loader, criterion, CFG['device'], num_classes=6, id2label=id2label)


    # --- WandB 통합 로깅 (정밀 모니터링) ---
    log_dict = {
        'epoch': epoch,
        'Loss/Train': avg_train_loss,
        'Loss/Val': avg_val_loss,
        'Metrics/mIoU': avg_val_miou,
        'Metrics/mBoU': avg_val_mbou
    }

    # 클래스별 상세 IoU 기록
    for i, (cat_iou, b_iou) in enumerate(zip(mean_cat_ious, mean_bound_ious)):
        cls_name = id2label[i]
        log_dict[f'Class_IoU/{cls_name}'] = cat_iou
        log_dict[f'Class_Boundary_IoU/{cls_name}'] = b_iou

    wandb.log(log_dict)



    print(f"📝 Epoch [{epoch+1}/{CFG['epochs']}]")
    print(f"   Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"   ✨ Val mIoU: {avg_val_miou:.4f} | mBoU: {avg_val_mbou:.4f}")

    # --------------------------------------------------------------------------
    # 💾 3. 이중 자동 저장 (Dual Auto-Save)
    # --------------------------------------------------------------------------
    # [기준 1] mIoU 챔피언 (가장 똑똑한 모델)
    if avg_val_miou > best_miou:
        print(f"   🏆 New Best mIoU! ({best_miou:.4f} -> {avg_val_miou:.4f}) Saving...")
        best_miou = avg_val_miou
        best_miou_epoch = epoch + 1
        torch.save(model.state_dict(), os.path.join(save_dir, "segformer_best_miou.pth"))

    # [기준 2] mBoU 챔피언 (테두리 장인)
    if avg_val_mbou > best_mbou:
        print(f"   🎨 New Best mBoU! ({best_mbou:.4f} -> {avg_val_mbou:.4f}) Saving...")
        best_mbou = avg_val_mbou
        torch.save(model.state_dict(), os.path.join(save_dir, "segformer_best_mbou.pth"))

    # 0.5 수준: "기반은 닦았으나, 고득점 문제는 포기한 상태"
    # 0.1 수준: "실전에 투입해볼 만한 우등생"
    # 0.01 수준: "완벽, 혹은 너무 과한 공부(과적합)"
    print("-" * 50)

print(f"\n🎉 Training Complete!")
print(f"   👑 Overall Best mIoU: {best_miou:.4f} (at Epoch {best_miou_epoch})")
print(f"   🖌️ Overall Best mBoU: {best_mbou:.4f}")
# 마지막 상태도 저장
torch.save(model.state_dict(), os.path.join(save_dir, "segformer_last.pth"))

wandb.finish()



🚀 SegFormer-B2 Training Start with Dual Auto-Save Strategy...


Class_Boundary_IoU/Lane Mark,▁▂▂█
Class_Boundary_IoU/Moveable,▁▂█▆
Class_Boundary_IoU/My bike,▃▁▆█
Class_Boundary_IoU/Rider,▁▂██
Class_Boundary_IoU/Road,▄▁█▇
Class_Boundary_IoU/Undrivable,▁▂▅█
Class_IoU/Lane Mark,▁▅▄█
Class_IoU/Moveable,▁▃▇█
Class_IoU/My bike,▁▅▇█
Class_IoU/Rider,▁▁█▇
+7,...


Epoch 1 [Train]:   0%|          | 0/18 [00:00<?, ?it/s]


[Validation Report]
  Category-specific IoU:
    - Rider: 0.1162
    - My bike: 0.4940
    - Moveable: 0.0763
    - Lane Mark: 0.0682
    - Road: 0.3403
    - Undrivable: 0.2445

  Boundary IoU (Details):
    - Rider: 0.0052
    - My bike: 0.0235
    - Moveable: 0.0012
    - Lane Mark: 0.0316
    - Road: 0.0140
    - Undrivable: 0.0008

  --> mIoU: 0.2233 | mBoU: 0.0127

📝 Epoch [1/20]
   Train Loss: 1.5866 | Val Loss: 1.4903
   ✨ Val mIoU: 0.2233 | mBoU: 0.0127
   🏆 New Best mIoU! (0.0000 -> 0.2233) Saving...
   🎨 New Best mBoU! (0.0000 -> 0.0127) Saving...
--------------------------------------------------


Epoch 2 [Train]:   0%|          | 0/18 [00:00<?, ?it/s]


[Validation Report]
  Category-specific IoU:
    - Rider: 0.1832
    - My bike: 0.5457
    - Moveable: 0.1428
    - Lane Mark: 0.1242
    - Road: 0.4332
    - Undrivable: 0.5524

  Boundary IoU (Details):
    - Rider: 0.0107
    - My bike: 0.0128
    - Moveable: 0.0052
    - Lane Mark: 0.0346
    - Road: 0.0233
    - Undrivable: 0.0072

  --> mIoU: 0.3302 | mBoU: 0.0156

📝 Epoch [2/20]
   Train Loss: 1.2212 | Val Loss: 1.2346
   ✨ Val mIoU: 0.3302 | mBoU: 0.0156
   🏆 New Best mIoU! (0.2233 -> 0.3302) Saving...
   🎨 New Best mBoU! (0.0127 -> 0.0156) Saving...
--------------------------------------------------


Epoch 3 [Train]:   0%|          | 0/18 [00:00<?, ?it/s]


[Validation Report]
  Category-specific IoU:
    - Rider: 0.3846
    - My bike: 0.5911
    - Moveable: 0.2713
    - Lane Mark: 0.1268
    - Road: 0.5714
    - Undrivable: 0.7991

  Boundary IoU (Details):
    - Rider: 0.0181
    - My bike: 0.0178
    - Moveable: 0.0287
    - Lane Mark: 0.0266
    - Road: 0.0330
    - Undrivable: 0.0364

  --> mIoU: 0.4574 | mBoU: 0.0268

📝 Epoch [3/20]
   Train Loss: 1.0066 | Val Loss: 1.0433
   ✨ Val mIoU: 0.4574 | mBoU: 0.0268
   🏆 New Best mIoU! (0.3302 -> 0.4574) Saving...
   🎨 New Best mBoU! (0.0156 -> 0.0268) Saving...
--------------------------------------------------


Epoch 4 [Train]:   0%|          | 0/18 [00:00<?, ?it/s]

### 🛠️ 주요 평가 항목
- **mIoU** (Mean Intersection over Union):
  - **Category-specific IoU**
  - **Boundary IoU**
- **실시간성 및 하드웨어 지표**
  - **Model Parameters**
  - **MACs** Multiply-Accumulate Operations
    - y = wx + b 에서 wx + b를 1MAC 이라고 한다.
  - **GFLOPs** Giga Floating Point Operations
    -  모델을 한 번 실행(Forward Pass)할 때 필요한 총 부동 소수점 연산량
    - 보통 1MAC = 2FLOPs
  - **Average Inference Latency**
  - **Frames Per Second (FPS)**
- **Safety-critical Metrics**

- TODO
  - 테스트 이미지로 평가할 것 ( 현재는 전체 이미지 SET 사용 )

In [ ]:
# TODO : requirement.txt로 보내자
import sys
!{sys.executable} -m pip install thop

print("thop installed successfully.")

thop installed successfully.


In [ ]:
import time
from thop import profile

print("🚀 Starting Model Profiling...")

# 2. 모델을 위한 더미 입력 텐서 생성
dummy_input = torch.randn(1, 3, CFG['img_size'][0], CFG['img_size'][1]).to(CFG['device'])

# 3. 더미 입력을 사용하여 모델의 MACs 및 매개변수(params)를 계산하기 위해 thop.profile 사용. verbose=False로 설정.
macs, params = profile(model, inputs=(dummy_input,), verbose=False)

# 4. 계산된 MACs를 1e9로 나누어 GFLOPs로 변환.
gflops = macs / 1e9

print(f"Model Parameters (M): {params / 1e6:.2f}")
print(f"MACs (G): {macs / 1e9:.2f}")
print(f"GFLOPs: {gflops:.2f}")

# 5. 지연 시간 및 FPS 측정을 위한 변수 초기화.
num_warmup_runs = 10
num_inference_runs = 100
total_latency = 0.0

# 모델을 평가 모드로 설정
model.eval()

# 6. 워밍업 실행 수행
print(f"Performing {num_warmup_runs} warm-up runs...")
with torch.no_grad():
    for _ in range(num_warmup_runs):
        _ = model(dummy_input)

# 7. 추론 지연 시간 측정
print(f"Measuring latency over {num_inference_runs} inference runs...")
with torch.no_grad():
    for _ in range(num_inference_runs):
        start_time = time.perf_counter()
        _ = model(dummy_input)
        end_time = time.perf_counter()
        total_latency += (end_time - start_time)

# 8. 평균 지연 시간(ms) 계산
average_latency_ms = (total_latency / num_inference_runs) * 1000

# 9. FPS 계산
fps = 1000 / average_latency_ms

print(f"Average Inference Latency: {average_latency_ms:.2f} ms")
print(f"Frames Per Second (FPS): {fps:.2f}")
print("Model Profiling Complete.")

🚀 Starting Model Profiling...
Model Parameters (M): 27.35
MACs (G): 26.83
GFLOPs: 26.83
Performing 10 warm-up runs...
Measuring latency over 100 inference runs...
Average Inference Latency: 83.82 ms
Frames Per Second (FPS): 11.93
Model Profiling Complete.


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


class SafetyEvalMetrics:
    def __init__(self, num_classes, class_names):
        self.num_classes = num_classes
        self.class_names = class_names
        self.confusion_matrix = np.zeros((num_classes, num_classes))

    def update(self, preds, gts):
        """
        한 배치(Batch) 또는 한 프레임의 결과를 누적합니다.
        preds, gts: [Batch, H, W] 형태의 Tensor 또는 Numpy
        """
        preds = preds.flatten()
        gts = gts.flatten()

        # 유효하지 않은 타겟 값 제외 (예: 패딩 또는 ignore_index가 포함된 경우)
        valid_mask = (gts >= 0) & (gts < self.num_classes)
        preds = preds[valid_mask]
        gts = gts[valid_mask]

        # sklearn의 confusion_matrix를 사용하여 매트릭스 업데이트
        # labels 인자를 명시적으로 제공하여 모든 클래스가 포함되도록 보장
        new_cm = confusion_matrix(gts, preds, labels=range(self.num_classes))
        self.confusion_matrix += new_cm

    def plot_confusion_matrix(self, normalize=True):
        """
        혼동 행렬 시각화
        """
        cm = self.confusion_matrix
        if normalize:
            # 행(Actual) 기준 정규화: 해당 클래스가 실제 무엇으로 예측되었는지 비율 확인
            # 0으로 나누는 오류 방지를 위해 1e-9 추가
            cm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-9)

        plt.figure(figsize=(8, 5))
        sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d",
                    cmap="Blues", xticklabels=self.class_names, yticklabels=self.class_names)

        plt.title('Road Scene Segmentation: Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('Ground Truth Label')
        plt.show()

    def analyze_safety_risks(self):
        """
        치명적 오분류(Safety-Critical) 집중 분석
        """
        cm_norm = self.confusion_matrix.astype('float') / (self.confusion_matrix.sum(axis=1)[:, np.newaxis] + 1e-9)

        print("\n=== [Safety-Critical Analysis] ===")

        try:
            rider_idx = self.class_names.index('Rider')
            mybike_idx = self.class_names.index('My bike')
            moveable_idx = self.class_names.index('Moveable')
            lanemark_idx = self.class_names.index('Lane Mark')
            road_idx = self.class_names.index('Road')
            undrivable_idx = self.class_names.index('Undrivable')

            print("--- 고위험 ---")
            # Case 1: Undrivable (주행 불가 영역)이 Road (주행 가능 도로)로 오분류된 경우
            # 실제 장애물이나 위험 지역을 도로로 인식하여 충돌 위험이 매우 높음.
            undrivable_as_road = cm_norm[undrivable_idx, road_idx]
            print(f"1. 즉각적인 충돌 위험 - 'Undrivable' 영역의 {undrivable_as_road:.2%}가 'Road'로 오분류 됨.")

            # Case 2: Moveable (이동 물체)이 Undrivable (주행 불가 영역)로 오분류된 경우
            # 다른 차량이나 보행자 등 중요한 이동 장애물을 감지하지 못하고 통행 불가 영역으로 표시했음을 의미.
            missed_moveable_as_undrivable = cm_norm[moveable_idx, undrivable_idx]
            print(f"2. 이동 물체 충돌 위험 - 'Moveable' 객체의 {missed_moveable_as_undrivable:.2%}가 'Undrivable'로 오분류 됨.")

            # Case 3: Moveable (이동 물체)이 Road (주행 가능 도로)로 오분류된 경우
            # 다른 차량이나 보행자 등 중요한 이동 장애물을 감지하지 못하고 안전한 주행 가능 영역으로 표시했음을 의미.
            missed_moveable_as_road = cm_norm[moveable_idx, road_idx]
            print(f"3. 이동 물체 충돌 위험 - 'Moveable' 객체의 {missed_moveable_as_road:.2%}가 'Road'로 오분류 됨.")

            # Case 4: Rider (탑승자)가 Undrivable (주행 불가 영역)로 오분류된 경우
            # 자율주행 시스템이 탑승자(자신)를 장애물로 인식하여 불필요한 회피 기동이나 정지를 유발할 수 있음.
            rider_as_undrivable = cm_norm[rider_idx, undrivable_idx]
            print(f"4. 시스템 오작동 유발 - 'Rider' 객체의 {rider_as_undrivable:.2%}가 'Undrivable'로 오분류 됨.")

            print("\n--- 중위험 ---")
            # Case 5: Road (주행 가능 도로)가 Undrivable (주행 불가 영역)로 오분류된 경우
            # 주행 가능한 도로를 통행 불가능하다고 간주하여 잘못된 경로 계획이나 불필요한 제동으로 이어질 수 있음.
            road_to_undrivable = cm_norm[road_idx, undrivable_idx]
            print(f"5. 경로 계획 오류 - 'Road' 영역의 {road_to_undrivable:.2%}가 'Undrivable'로 오분류 됨.")

            # Case 6: Lane Mark (차선)가 Road (주행 가능 도로)로 오분류된 경우
            # 차선이 도로의 일부로 인식되어 차선 유지 보조 시스템의 오작동을 유발하거나 차선 이탈을 방지하지 못할 수 있음.
            lanemark_as_road = cm_norm[lanemark_idx, road_idx]
            print(f"6. 차선 유지 보조 오류 - 'Lane Mark' 영역의 {lanemark_as_road:.2%}가 'Road'로 오분류 됨.")

            print("\n--- 낮은-중간 위험 ---")
            # Case 7: Rider (탑승자)가 My bike (내 오토바이)로 오분류된 경우
            # 시스템의 자기 인식에 혼란을 주지만, 직접적인 충돌 위험은 낮음. 정밀한 자율 주행에는 영향.
            rider_as_mybike = cm_norm[rider_idx, mybike_idx]
            print(f"7. 자기 인식 오류 - 'Rider' 객체의 {rider_as_mybike:.2%}가 'My bike'로 오분류 됨.")

        except ValueError as e:
            print(f"오류: 데이터셋의 클래스 이름을 확인하세요: {e}")


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

print("🚀 Starting Safety Evaluation...")

# Instantiate SafetyEvalMetrics
class_names = [name for i, name in sorted(id2label.items())]
evaluator = SafetyEvalMetrics(num_classes=CFG_EVAL['num_classes'], class_names=class_names)

# Evaluate the model on the validation set to update the confusion matrix
model.eval()
with torch.no_grad():
    for images, masks in val_loader:
        X = images.to(CFG['device']).contiguous()
        y = masks.to(CFG['device']).contiguous()

        outputs = model(X).logits
        upsampled_logits = nn.functional.interpolate(
            outputs,
            size=y.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        preds = upsampled_logits.argmax(dim=1)
        evaluator.update(preds.cpu().numpy(), y.cpu().numpy())

# Plot the confusion matrix
evaluator.plot_confusion_matrix(normalize=True)
evaluator.analyze_safety_risks()

## 🚦 Grad-CAM: 모델 의사결정 근거 시각화

**1. 분석 대상:** `segformer_best_miou.pth` (최고 성능 가중치 로드)
**2. 선정 이유:** **mIoU**가 가장 높은 모델의 '판단 논리'를 정성적으로 검증하기 위함

**3. 기술적 요점:**
* **Target:** Encoder Stage 4 MLP (최종 특징 추출 단계)
* **Method:** Transformer 시퀀스 데이터를 2D 피처맵($12 \times 21$)으로 복원 시각화
* **Layout:** 원본 대비 클래스별 히트맵 1행 2열 배치 (무작위 샘플링)



**💡 요약:** "가장 똑똑한 모델(Best mIoU)이 밤길 객체를 정확한 근거로 식별하는지 시각적으로 증명합니다."

In [ ]:
# !pip install grad-cam
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import SemanticSegmentationTarget

# -------------------------------------------------------------------------
# [ 핵심 ] 1. 최고의 성능을 낸 mIoU 모델(.pth) 불러오기
# -------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint_path = "./checkpoints/segformer_best_miou.pth"

# 모델에 학습된 가중치 주입 (이 과정이 있어야 '지능'이 탑재됩니다)
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"✅ 학습 완료된 베스트 모델 로드 성공: {checkpoint_path}")
else:
    print(f"⚠️ 경고: {checkpoint_path} 파일을 찾을 수 없습니다. 현재 메모리의 모델을 사용합니다.")

# 2. Grad-CAM용 Wrapper 및 Reshape 함수 설정
class SegformerModelWrapper(nn.Module):
    def __init__(self, model):
        super(SegformerModelWrapper, self).__init__()
        self.model = model
    def forward(self, x):
        return self.model(x).logits

def segformer_reshape_transform(tensor, height=12, width=21):
    result = tensor.reshape(tensor.size(0), height, width, -1)
    result = result.transpose(2, 3).transpose(1, 2)
    return result

wrapped_model = SegformerModelWrapper(model).to(device)
wrapped_model.eval()

# 3. Grad-CAM 객체 생성 (Stage 4 MLP 타겟)
target_layers = [wrapped_model.model.segformer.encoder.block[3][1].mlp]
cam = GradCAM(model=wrapped_model, 
              target_layers=target_layers, 
              reshape_transform=segformer_reshape_transform)

# 4. 무작위 사진 선택 (매번 실행 시마다 새로운 데이터)
random_idx = random.randint(0, len(val_dataset) - 1)
image, mask = val_dataset[random_idx]
input_tensor = image.unsqueeze(0).to(device)
input_tensor.requires_grad = True

# 5. 배경 이미지 준비
rgb_img = input_tensor[0].detach().cpu().permute(1, 2, 0).numpy()
rgb_img = (rgb_img - rgb_img.min()) / (rgb_img.max() - rgb_img.min())

# ==========================================
# 6. 4행 2열(총 8칸) 그리드 설정 및 출력
# ==========================================
num_classes = 6
fig, axes = plt.subplots(4, 2, figsize=(20, 25))
axes = axes.flatten()

print(f"📸 분석 시작: Best mIoU 모델을 사용해 {random_idx}번 이미지를 분석합니다.")

# [첫 번째 칸] 원본 이미지
axes[0].imshow(rgb_img)
axes[0].set_title(f"Original Image (Idx: {random_idx})", fontsize=18, fontweight='bold')
axes[0].axis('off')

# [두 번째~일곱 번째 칸] 각 클래스별 Grad-CAM 계산 및 합성
output_h, output_w = input_tensor.shape[2] // 4, input_tensor.shape[3] // 4
mask_float = np.ones((output_h, output_w), dtype=np.float32)

for i in range(num_classes):
    class_name = id2label[i]
    targets = [SemanticSegmentationTarget(category=i, mask=mask_float)]
    
    # Grad-CAM 생성
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    
    # 그리드 배치
    axes[i+1].imshow(visualization)
    axes[i+1].set_title(f"Class {i}: {class_name.upper()}", fontsize=18, color='darkred', fontweight='bold')
    axes[i+1].axis('off')

# [여덟 번째 칸] 정보 표시
axes[7].axis('off')
axes[7].text(0.5, 0.5, f"Analysis Model: Best mIoU\nData Index: {random_idx}\nTarget: Multi-Class Logic", 
             ha='center', va='center', fontsize=16, color='gray', style='italic', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"✅ 모든 시각화가 완료되었습니다. (사용한 모델: {checkpoint_path})")